In [ ]:
!pip install polars numpy implicit tqdm pyarrow

In [1]:
!pip install pandas==1.3.1


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import polars as pl
import numpy as np
from tqdm import tqdm
from scipy.sparse import csr_matrix
import implicit
import pandas as pd
RANDOM_STATE = 42
N_PREDICTIONS = 100
from pathlib import Path
data_path = Path('data')
train_logs = data_path / 'hh_recsys_train_hh.pq'
test_logs = data_path / 'hh_recsys_test_hh.pq'
vacancies_content = data_path / 'hh_recsys_vacancies.pq'
submission_sample = data_path / 'hh_recsys_sample.pq'

In [3]:
#pl.read_parquet(str(submission_sample))

In [250]:
train = pl.read_parquet(str(train_logs))
test = pl.read_parquet(str(test_logs))



##### Контент вакансий
До их разбора пока руки не дошли

In [5]:
#vacancies = pl.read_parquet(str(vacancies_content))
#vacancies

In [251]:
train = train.to_pandas()
test = test.to_pandas()
# vacancies = vacancies.to_pandas()

### EDA (комментарии в процессе)

In [8]:
train

,user_id,session_id,vacancy_id,action_type,action_dt
0,u_332060,s_28301374,"[v_2571684, v_488179, v_2389179, v_1393783, v_...","[2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, ...","[2023-11-01T00:40:58.105000000, 2023-11-01T00:..."
1,u_1057881,s_33868982,[v_665861],[2],[2023-11-01T00:23:51.452000000]
2,u_1036784,s_32474802,[v_2594840],[2],[2023-11-01T00:52:34.023000000]
3,u_786220,s_14060785,"[v_1473781, v_1622905, v_1621959, v_2289180, v...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, ...","[2023-11-01T00:58:20.793000000, 2023-11-01T01:..."
4,u_639152,s_23205986,"[v_695738, v_22433, v_1590524, v_502496, v_200...","[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, ...","[2023-11-01T01:14:20.828000000, 2023-11-01T00:..."
...,...,...,...,...,...
3463059,u_202578,s_8306993,[v_1499243],[2],[2023-11-13T21:01:41.813000000]
3463060,u_895531,s_7293998,[v_2041381],[2],[2023-11-14T17:51:22.169000000]
3463061,u_317562,s_9866576,[v_503192],[2],[2023-11-14T20:03:54.988000000]
3463062,u_225581,s_19724962,[v_2526106],[2],[2023-11-14T20:52:51.985000000]


#### Временные рамки лога, частота событий в день

In [230]:
train.sample(frac=0.05).action_dt.apply(lambda x: pd.to_datetime(x[0]).date()).value_counts()

2023-11-13    17180
2023-11-14    16779
2023-11-07    16727
2023-11-08    16266
2023-11-09    15641
2023-11-01    15413
2023-11-02    15088
2023-11-10    13396
2023-11-03    12236
2023-11-06     8577
2023-11-12     7285
2023-11-11     6411
2023-11-04     6173
2023-11-05     5981
Name: action_dt, dtype: int64

In [37]:
test.sample(frac=0.5).action_dt.apply(lambda x: pd.to_datetime(x[0]).date()).value_counts()

2023-11-20    8160
2023-11-21    7966
2023-11-16    7045
2023-11-15    6611
2023-11-17    5908
2023-11-19    3131
2023-11-18    2773
Name: action_dt, dtype: int64

In [11]:
train.user_id.value_counts()

count    882409.000000
mean          3.924557
std           5.444649
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max         828.000000
Name: user_id, dtype: float64

In [12]:
test.user_id.value_counts().describe()

count    83189.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: user_id, dtype: float64

In [25]:
train_vacancy_counts = train.explode('vacancy_id').vacancy_id.value_counts()
train_vacancy_counts.describe()

count    1.458353e+06
mean     1.475368e+01
std      5.082756e+01
min      1.000000e+00
25%      1.000000e+00
50%      4.000000e+00
75%      1.200000e+01
max      1.325400e+04
Name: vacancy_id, dtype: float64

In [17]:
train.explode('vacancy_id').user_id.value_counts().describe()

count    882409.00000
mean         24.38333
std         289.57993
min           1.00000
25%           2.00000
50%           7.00000
75%          22.00000
max      260689.00000
Name: user_id, dtype: float64

In [21]:
def get_user_vacancy_count(df):
    return df.explode('vacancy_id').user_id.value_counts()
train_user_vacancy_count = get_user_vacancy_count(train)
test_user_vacancy_count = get_user_vacancy_count(test)
train_user_vacancy_count.describe()

count    882409.00000
mean         24.38333
std         289.57993
min           1.00000
25%           2.00000
50%           7.00000
75%          22.00000
max      260689.00000
Name: user_id, dtype: float64

### Препроцессинг

сделал более мудреный препроц на классах, чтобы можно было одинаково обрабатывать train и тест и чтобы препроц был сразу из коробки обратимым - сначала нужно идшники перегнать из строк в инты для производительности, а для сабмита вернуть их к исходному виду.

In [252]:

class IdParser:
    def __init__(self, original_col, parsed_col, prefix):
        self.original_col = original_col
        self.parsed_col = parsed_col
        self.prefix = prefix
    def transform(self, df):
        df[self.original_col] = df[self.original_col].apply(self.id_parser)
        return df
    def inverse_transform(self, df):
        df[self.original_col] = df[self.original_col].apply(self.id_construct)
        return df
    def id_parser(self, user_id_str):
        return int(user_id_str[len(self.prefix):])
    def id_construct(self, id_):
        return f'{self.prefix}{id_}'

class IdArrayParser(IdParser):
    def id_parser(self, id_arr):
        return [super(IdArrayParser, self).id_parser(id_) for id_ in id_arr] 

    def id_construct(self, id_arr):
        return [super(IdArrayParser, self).id_construct(id_) for id_ in id_arr] 


class Pipeline:
    def __init__(self, stages):
        self.stages = stages
    def transform(self, x):
        for stage in self.stages:
            x = stage.transform(x)
        return x
    def inverse_transform(self, x):
        for stage in reversed(self.stages):
            x = stage.inverse_transform(x)
        return x
class ColumnRename:
    def __init__(self, col_mapping, skip_reverse=True):
        self.mapping = col_mapping
        self.skip_reverse = skip_reverse
        self.inverse_mapping = dict(((v,k) for k,v in col_mapping.items()))
    def transform(self, df):
        return df.drop(self.mapping.values(), axis=1).rename(self.mapping, axis=1)
    def inverse_transform(self, df):
        existing_keys = {k:v for k,v in self.inverse_mapping.items() if k in df.columns}
        return df.drop(self.inverse_mapping.values(), axis=1).rename(existing_keys, axis=1)
class SessionDt:
    def __init__(self, dt_arr_col):
        self.dt_col = dt_arr_col
    def transform(self, df):
        df['session_dt'] = df[self.dt_col].apply(lambda dts: dts[0])
        return df
    def inverse_transform(self, df):
        return df
als_preproc_pipeline = Pipeline([
    IdParser('user_id', 'user_id_int', 'u_'),
    IdArrayParser('vacancy_id', 'vacancy_arr', 'v_'), 
    SessionDt('action_dt'),
    #ColumnRename({'user_id_int':'user_id', 'vacancy_arr':'vacancy_id'})
])
train_val_split_date = '2023-11-14'
als_train = als_preproc_pipeline.transform(train)
# train/val сплит пока не готов
#als_val = als_train[als_train.session_dt >= train_val_split_date]
#als_train = als_train[als_train.session_dt < train_val_split_date]
als_test = als_preproc_pipeline.transform(test)

In [253]:
als_train

,user_id,session_id,vacancy_id,action_type,action_dt,session_dt
0,332060,s_28301374,"[2571684, 488179, 2389179, 1393783, 2608935, 1...","[2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, ...","[2023-11-01T00:40:58.105000000, 2023-11-01T00:...",2023-11-01 00:40:58.105
1,1057881,s_33868982,[665861],[2],[2023-11-01T00:23:51.452000000],2023-11-01 00:23:51.452
2,1036784,s_32474802,[2594840],[2],[2023-11-01T00:52:34.023000000],2023-11-01 00:52:34.023
3,786220,s_14060785,"[1473781, 1622905, 1621959, 2289180, 1473781, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, ...","[2023-11-01T00:58:20.793000000, 2023-11-01T01:...",2023-11-01 00:58:20.793
4,639152,s_23205986,"[695738, 22433, 1590524, 502496, 2001124, 1740...","[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, ...","[2023-11-01T01:14:20.828000000, 2023-11-01T00:...",2023-11-01 01:14:20.828
...,...,...,...,...,...,...
3463059,202578,s_8306993,[1499243],[2],[2023-11-13T21:01:41.813000000],2023-11-13 21:01:41.813
3463060,895531,s_7293998,[2041381],[2],[2023-11-14T17:51:22.169000000],2023-11-14 17:51:22.169
3463061,317562,s_9866576,[503192],[2],[2023-11-14T20:03:54.988000000],2023-11-14 20:03:54.988
3463062,225581,s_19724962,[2526106],[2],[2023-11-14T20:52:51.985000000],2023-11-14 20:52:51.985


In [254]:
als_preproc_pipeline.inverse_transform(als_train)

,user_id,session_id,vacancy_id,action_type,action_dt,session_dt
0,u_332060,s_28301374,"[v_2571684, v_488179, v_2389179, v_1393783, v_...","[2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, ...","[2023-11-01T00:40:58.105000000, 2023-11-01T00:...",2023-11-01 00:40:58.105
1,u_1057881,s_33868982,[v_665861],[2],[2023-11-01T00:23:51.452000000],2023-11-01 00:23:51.452
2,u_1036784,s_32474802,[v_2594840],[2],[2023-11-01T00:52:34.023000000],2023-11-01 00:52:34.023
3,u_786220,s_14060785,"[v_1473781, v_1622905, v_1621959, v_2289180, v...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, ...","[2023-11-01T00:58:20.793000000, 2023-11-01T01:...",2023-11-01 00:58:20.793
4,u_639152,s_23205986,"[v_695738, v_22433, v_1590524, v_502496, v_200...","[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, ...","[2023-11-01T01:14:20.828000000, 2023-11-01T00:...",2023-11-01 01:14:20.828
...,...,...,...,...,...,...
3463059,u_202578,s_8306993,[v_1499243],[2],[2023-11-13T21:01:41.813000000],2023-11-13 21:01:41.813
3463060,u_895531,s_7293998,[v_2041381],[2],[2023-11-14T17:51:22.169000000],2023-11-14 17:51:22.169
3463061,u_317562,s_9866576,[v_503192],[2],[2023-11-14T20:03:54.988000000],2023-11-14 20:03:54.988
3463062,u_225581,s_19724962,[v_2526106],[2],[2023-11-14T20:52:51.985000000],2023-11-14 20:52:51.985


In [255]:
als_test

,user_id,session_id,vacancy_id,action_type,action_dt,session_dt
0,482520,s_25018731,"[2597196, 1223061, 1223061]","[2, 2, 1]","[2023-11-19T12:03:13.089000000, 2023-11-19T12:...",2023-11-19 12:03:13.089
1,582132,s_481216,"[470400, 470400, 1530783]","[2, 1, 2]","[2023-11-21T15:39:47.981000000, 2023-11-21T15:...",2023-11-21 15:39:47.981
2,212584,s_16918781,"[1572055, 1572055, 1572055, 953153, 953153, 14...","[2, 3, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1]","[2023-11-16T08:41:47.031000000, 2023-11-16T08:...",2023-11-16 08:41:47.031
3,425177,s_17505104,"[1375331, 1922852]","[2, 2]","[2023-11-17T12:42:18.513000000, 2023-11-17T12:...",2023-11-17 12:42:18.513
4,700997,s_15528830,"[2152997, 2152997, 1217630]","[2, 1, 2]","[2023-11-16T17:22:53.530000000, 2023-11-16T17:...",2023-11-16 17:22:53.530
...,...,...,...,...,...,...
83184,499368,s_19158589,"[1716634, 2232327, 2232327, 2232327, 408610, 2...","[2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2]","[2023-11-18T03:15:15.530000000, 2023-11-18T03:...",2023-11-18 03:15:15.530
83185,1084868,s_24756162,"[395706, 2405231]","[2, 2]","[2023-11-15T18:06:16.550000000, 2023-11-15T18:...",2023-11-15 18:06:16.550
83186,1138032,s_25291467,"[1213925, 1213925, 2359832, 2359832]","[2, 1, 2, 1]","[2023-11-19T23:19:44.092000000, 2023-11-19T23:...",2023-11-19 23:19:44.092
83187,608102,s_18813020,"[933773, 933773, 2021127, 1508520, 2021127, 20...","[3, 2, 2, 2, 2, 1, 2]","[2023-11-16T15:40:09.408000000, 2023-11-16T15:...",2023-11-16 15:40:09.408


# Simple ALS

In [227]:
from abc import ABC, abstractmethod
from dataclasses import dataclass

from sklearn.preprocessing import LabelEncoder


# Пока эти классы используются чисто как документация, чтобы было понятно с чем работает модель
@dataclass
class UserItems:
    user_id: int
    item: list

@dataclass
class UserItemHistory(UserItems):
    action_type: list
    session_dt: list

@dataclass
class Predict(UserItems):
    score: list

# Все что нужно АЛС модели от датасета
class UsrHistory(ABC):
    @abstractmethod
    def get_unique_users(self):
        pass
    @abstractmethod
    def get_unique_items(self):
        pass
    @abstractmethod
    def get_per_item_interactions(self):
        pass
    
# обертка над пандасовским датасетом, чтобы проще писать рек модель
class PandasUsrHistory(UsrHistory):
    def __init__(self, data):
        self.data = data
    def get_unique_users(self):
        return self.data['user_id'].unique()
    def get_unique_items(self):
        return self.data['vacancy_id'].explode().unique()
    def get_per_item_interactions(self):
        return self.data[['user_id', 'vacancy_id', 'action_type']] \
        .rename({'vacancy_id':'item'}, axis=1) \
        .explode(['item', 'action_type'])

# интерфейс к мапперу действия юзера в вес АЛС
class ActionWeightMapper(ABC):
    @abstractmethod
    def transform(self, weight):
        pass
# маппер на основе словаря
class PandasDictWeightMapper(ActionWeightMapper):
    def __init__(self, mapping):
        self.action_weights = mapping
    def transform(self, actions):
        return actions.apply(lambda action: self.action_weights[action])

# класс, который берет на себя ремап из хешей int64 в последовательные идшники 
# и собирает на них датасет для матричной факторизации
# так же умеет инференсить рекомы для юзера из исходного датасета
class AlsDataset:
    def __init__(self, mf_model, action_weights_mapper):
        self.mf_model = mf_model
        self.action_weights = action_weights_mapper
    
    def prepare_dataset(self, user_histories: UsrHistory):
        #pairs = df[['user_id', 'item', 'action_type']].explode(['item', 'action_type'])
        self.user_encoder = LabelEncoder().fit(user_histories.get_unique_users())
        self.item_encoder = LabelEncoder().fit(user_histories.get_unique_items())
        
        pairs = user_histories.get_per_item_interactions()
        users = self.user_encoder.transform(pairs['user_id'])
        vacancies = self.item_encoder.transform(pairs['item'])
        preferences = self.action_weights.transform(pairs['action_type'])
        uv_mat = csr_matrix((preferences, (users, vacancies)))
        self.uv_mat = uv_mat
        return uv_mat
    
    def recommend(self, user, topk=5):
        cuser = self.user_encoder.transform([user])
        # почему-то items, scores распаковываются от als_model в массив в массиве
        # пока просто живу с этим
        items, scores = self.mf_model.recommend(cuser, self.uv_mat[cuser], N=topk)
        items = self.item_encoder.inverse_transform(items[0])
        return Predict(user, items, scores[0])
    


als_dataset_pair_gen = AlsDataset(
    als_model, 
    PandasDictWeightMapper({
            1: 4.0,
            2: 1.0,
            3: 2.0
        })
)


als_uv_mat_train = als_dataset_pair_gen.prepare_dataset(PandasUsrHistory(als_train))#.transform(als_train)


In [140]:

als_model = implicit.als.AlternatingLeastSquares(
    factors=150,
    random_state=RANDOM_STATE,
    iterations=5,
    calculate_training_loss=True,
    regularization=0.001
)
   

als_model.fit(als_uv_mat_train)

  0%|          | 0/5 [00:00<?, ?it/s]

In [229]:
als_dataset_pair_gen.recommend(456512)

Predict(user_id=456512, item=array([11662, 8338, 36723, 1703, 14949], dtype=object), score=array([0.00740471, 0.0048339 , 0.00379048, 0.00366266, 0.00338457],
      dtype=float32))

In [66]:
list(als_dataset_pair_gen.user2idx.keys())[:10]

[332060,
 1057881,
 1036784,
 786220,
 639152,
 456512,
 584261,
 733524,
 1114237,
 407319]

In [20]:
als_model.user_factors.shape, als_model.item_factors.shape

((882409, 150), (1458353, 150))

In [141]:
test_users = als_test['user_id'].to_list()
test_vacancies = als_test['vacancy_id']

In [148]:
predictions = []

for user, vacs in tqdm(zip(test_users, test_vacancies), total=len(test_users)):
    if user not in als_dataset_pair_gen.user2idx:
        items = vacs
        scores = [0 for i in vacs]
        predict = Predict(user, items, scores)
    else:        
        predict = als_dataset_pair_gen.recommend(user)
    predictions.append(predict)
    
predicts = pd.DataFrame(predictions)
predictions = als_dataset_pipeline.inverse_transform(predicts)

100%|██████████| 83189/83189 [1:00:46<00:00, 22.81it/s]


AttributeError: 'AlsDataset' object has no attribute 'reverse_transform'

In [156]:
test

,user_id,session_id,vacancy_id,action_type,action_dt,user_id_int,vacancy_arr
0,u_482520,s_25018731,"[v_2597196, v_1223061, v_1223061]","[2, 2, 1]","[2023-11-19T12:03:13.089000000, 2023-11-19T12:...",482520,"[2597196, 1223061, 1223061]"
1,u_582132,s_481216,"[v_470400, v_470400, v_1530783]","[2, 1, 2]","[2023-11-21T15:39:47.981000000, 2023-11-21T15:...",582132,"[470400, 470400, 1530783]"
2,u_212584,s_16918781,"[v_1572055, v_1572055, v_1572055, v_953153, v_...","[2, 3, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1]","[2023-11-16T08:41:47.031000000, 2023-11-16T08:...",212584,"[1572055, 1572055, 1572055, 953153, 953153, 14..."
3,u_425177,s_17505104,"[v_1375331, v_1922852]","[2, 2]","[2023-11-17T12:42:18.513000000, 2023-11-17T12:...",425177,"[1375331, 1922852]"
4,u_700997,s_15528830,"[v_2152997, v_2152997, v_1217630]","[2, 1, 2]","[2023-11-16T17:22:53.530000000, 2023-11-16T17:...",700997,"[2152997, 2152997, 1217630]"
...,...,...,...,...,...,...,...
83184,u_499368,s_19158589,"[v_1716634, v_2232327, v_2232327, v_2232327, v...","[2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2]","[2023-11-18T03:15:15.530000000, 2023-11-18T03:...",499368,"[1716634, 2232327, 2232327, 2232327, 408610, 2..."
83185,u_1084868,s_24756162,"[v_395706, v_2405231]","[2, 2]","[2023-11-15T18:06:16.550000000, 2023-11-15T18:...",1084868,"[395706, 2405231]"
83186,u_1138032,s_25291467,"[v_1213925, v_1213925, v_2359832, v_2359832]","[2, 1, 2, 1]","[2023-11-19T23:19:44.092000000, 2023-11-19T23:...",1138032,"[1213925, 1213925, 2359832, 2359832]"
83187,u_608102,s_18813020,"[v_933773, v_933773, v_2021127, v_1508520, v_2...","[3, 2, 2, 2, 2, 1, 2]","[2023-11-16T15:40:09.408000000, 2023-11-16T15:...",608102,"[933773, 933773, 2021127, 1508520, 2021127, 20..."


In [244]:
predicts.rename({'items':'vacancy_id'}, axis=1)

,user_id,vacancy_id,score
0,482520,"[2650515, 367301, 2710612, 1911300, 1768044, 1...","[0.08100045, 0.07667684, 0.07286401, 0.0714995..."
1,582132,"[1798396, 1347362, 2586135, 2126606, 1160600, ...","[0.032924023, 0.03286297, 0.03228987, 0.031543..."
2,212584,"[1572055, 1572055, 1572055, 953153, 953153, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,425177,"[2650515, 367301, 2710612, 1911300, 2145060, 1...","[0.0051576267, 0.004877838, 0.0046498175, 0.00..."
4,700997,"[1431178, 2395102, 1480870, 1161228, 1643983, ...","[0.25589028, 0.2326155, 0.22665058, 0.17441487..."
...,...,...,...
83184,499368,"[1716634, 2232327, 2232327, 2232327, 408610, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
83185,1084868,"[580809, 1350249, 2678794, 497984, 1624074, 18...","[0.015581891, 0.013927748, 0.0124956425, 0.011..."
83186,1138032,"[1213925, 1213925, 2359832, 2359832]","[0, 0, 0, 0]"
83187,608102,"[1075856, 497046, 186121, 1698351, 2595597, 97...","[0.00031002215, 0.00028363752, 0.00027466496, ..."


In [243]:
als_dataset_pipeline.inverse_transform(predicts.rename({'items':'vacancy_id'}, axis=1))

,user_id_int,vacancy_arr,vacancy_id,user_id
0,482520,"[2650515, 367301, 2710612, 1911300, 1768044, 1...","[v_2650515, v_367301, v_2710612, v_1911300, v_...",u_482520
1,582132,"[1798396, 1347362, 2586135, 2126606, 1160600, ...","[v_1798396, v_1347362, v_2586135, v_2126606, v...",u_582132
2,212584,"[1572055, 1572055, 1572055, 953153, 953153, 14...","[v_1572055, v_1572055, v_1572055, v_953153, v_...",u_212584
3,425177,"[2650515, 367301, 2710612, 1911300, 2145060, 1...","[v_2650515, v_367301, v_2710612, v_1911300, v_...",u_425177
4,700997,"[1431178, 2395102, 1480870, 1161228, 1643983, ...","[v_1431178, v_2395102, v_1480870, v_1161228, v...",u_700997
...,...,...,...,...
83184,499368,"[1716634, 2232327, 2232327, 2232327, 408610, 2...","[v_1716634, v_2232327, v_2232327, v_2232327, v...",u_499368
83185,1084868,"[580809, 1350249, 2678794, 497984, 1624074, 18...","[v_580809, v_1350249, v_2678794, v_497984, v_1...",u_1084868
83186,1138032,"[1213925, 1213925, 2359832, 2359832]","[v_1213925, v_1213925, v_2359832, v_2359832]",u_1138032
83187,608102,"[1075856, 497046, 186121, 1698351, 2595597, 97...","[v_1075856, v_497046, v_186121, v_1698351, v_2...",u_608102


In [157]:
predictions = als_dataset_pipeline.inverse_transform(predicts.rename({'items':'vacancy_id'}, axis=1))

In [158]:
predictions

,user_id_int,vacancy_arr,vacancy_id,user_id
0,482520,"[2650515, 367301, 2710612, 1911300, 1768044, 1...","[v_2650515, v_367301, v_2710612, v_1911300, v_...",u_482520
1,582132,"[1798396, 1347362, 2586135, 2126606, 1160600, ...","[v_1798396, v_1347362, v_2586135, v_2126606, v...",u_582132
2,212584,"[1572055, 1572055, 1572055, 953153, 953153, 14...","[v_1572055, v_1572055, v_1572055, v_953153, v_...",u_212584
3,425177,"[2650515, 367301, 2710612, 1911300, 2145060, 1...","[v_2650515, v_367301, v_2710612, v_1911300, v_...",u_425177
4,700997,"[1431178, 2395102, 1480870, 1161228, 1643983, ...","[v_1431178, v_2395102, v_1480870, v_1161228, v...",u_700997
...,...,...,...,...
83184,499368,"[1716634, 2232327, 2232327, 2232327, 408610, 2...","[v_1716634, v_2232327, v_2232327, v_2232327, v...",u_499368
83185,1084868,"[580809, 1350249, 2678794, 497984, 1624074, 18...","[v_580809, v_1350249, v_2678794, v_497984, v_1...",u_1084868
83186,1138032,"[1213925, 1213925, 2359832, 2359832]","[v_1213925, v_1213925, v_2359832, v_2359832]",u_1138032
83187,608102,"[1075856, 497046, 186121, 1698351, 2595597, 97...","[v_1075856, v_497046, v_186121, v_1698351, v_2...",u_608102


In [163]:
pl.from_pandas(test)

user_id,session_id,vacancy_id,action_type,action_dt,user_id_int,vacancy_arr,predictions
str,str,list[str],list[i64],list[datetime[ns]],i64,list[i64],list[str]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]",482520,"[2597196, 1223061, 1223061]","[""v_2650515"", ""v_367301"", … ""v_2385266""]"
"""u_582132""","""s_481216""","[""v_470400"", ""v_470400"", ""v_1530783""]","[2, 1, 2]","[2023-11-21 15:39:47.981, 2023-11-21 15:43:57.620, 2023-11-21 15:44:37.982]",582132,"[470400, 470400, 1530783]","[""v_1798396"", ""v_1347362"", … ""v_1752071""]"
"""u_212584""","""s_16918781""","[""v_1572055"", ""v_1572055"", … ""v_1329494""]","[2, 3, … 1]","[2023-11-16 08:41:47.031, 2023-11-16 08:45:21.403, … 2023-11-16 09:05:27.379]",212584,"[1572055, 1572055, … 1329494]","[""v_1572055"", ""v_1572055"", … ""v_1329494""]"
"""u_425177""","""s_17505104""","[""v_1375331"", ""v_1922852""]","[2, 2]","[2023-11-17 12:42:18.513, 2023-11-17 12:43:52.744]",425177,"[1375331, 1922852]","[""v_2650515"", ""v_367301"", … ""v_877535""]"
"""u_700997""","""s_15528830""","[""v_2152997"", ""v_2152997"", ""v_1217630""]","[2, 1, 2]","[2023-11-16 17:22:53.530, 2023-11-16 17:23:19.885, 2023-11-16 17:23:33.472]",700997,"[2152997, 2152997, 1217630]","[""v_1431178"", ""v_2395102"", … ""v_1333361""]"
"""u_687201""","""s_11187326""","[""v_2602959"", ""v_549337"", … ""v_212060""]","[2, 2, … 1]","[2023-11-21 14:41:30.546, 2023-11-21 15:00:25.506, … 2023-11-21 15:24:56.271]",687201,"[2602959, 549337, … 212060]","[""v_1465910"", ""v_1586766"", … ""v_758446""]"
"""u_573219""","""s_27394215""","[""v_1836184"", ""v_2733253"", … ""v_2297339""]","[1, 2, … 2]","[2023-11-21 18:48:43.424, 2023-11-21 18:49:12.552, … 2023-11-21 19:48:55.704]",573219,"[1836184, 2733253, … 2297339]","[""v_701974"", ""v_1081658"", … ""v_2337447""]"
"""u_1082782""","""s_30341164""","[""v_1026036"", ""v_1026036"", … ""v_206700""]","[2, 1, … 2]","[2023-11-17 02:54:19.561, 2023-11-17 02:54:50.486, … 2023-11-17 03:21:22.008]",1082782,"[1026036, 1026036, … 206700]","[""v_460169"", ""v_964765"", … ""v_1346336""]"
"""u_81444""","""s_1909964""","[""v_2497384"", ""v_1480870"", … ""v_1500697""]","[2, 2, … 1]","[2023-11-15 12:38:34.579, 2023-11-15 13:03:41.360, … 2023-11-15 13:05:15.916]",81444,"[2497384, 1480870, … 1500697]","[""v_1431178"", ""v_2395102"", … ""v_339861""]"


In [168]:
predictions['user_id vacancy_id'.split()].rename({'vacancy_id':'predictions'},axis=1)

,user_id,predictions
0,u_482520,"[v_2650515, v_367301, v_2710612, v_1911300, v_..."
1,u_582132,"[v_1798396, v_1347362, v_2586135, v_2126606, v..."
2,u_212584,"[v_1572055, v_1572055, v_1572055, v_953153, v_..."
3,u_425177,"[v_2650515, v_367301, v_2710612, v_1911300, v_..."
4,u_700997,"[v_1431178, v_2395102, v_1480870, v_1161228, v..."
...,...,...
83184,u_499368,"[v_1716634, v_2232327, v_2232327, v_2232327, v..."
83185,u_1084868,"[v_580809, v_1350249, v_2678794, v_497984, v_1..."
83186,u_1138032,"[v_1213925, v_1213925, v_2359832, v_2359832]"
83187,u_608102,"[v_1075856, v_497046, v_186121, v_1698351, v_2..."


In [170]:
test = test.merge(predictions['user_id vacancy_id'.split()].rename({'vacancy_id':'predictions'},axis=1),
                 on='user_id')
pl.from_pandas(test).select(['user_id', 'session_id', 'predictions']).write_parquet('als_submission.pq')

In [32]:
test

user_id,session_id,vacancy_id,action_type,action_dt,predictions
str,str,list[str],list[i64],list[datetime[ns]],list[str]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]","[""v_2650515"", ""v_367301"", … ""v_2337447""]"
"""u_582132""","""s_481216""","[""v_470400"", ""v_470400"", ""v_1530783""]","[2, 1, 2]","[2023-11-21 15:39:47.981, 2023-11-21 15:43:57.620, 2023-11-21 15:44:37.982]","[""v_1347362"", ""v_1798396"", … ""v_726002""]"
"""u_212584""","""s_16918781""","[""v_1572055"", ""v_1572055"", … ""v_1329494""]","[2, 3, … 1]","[2023-11-16 08:41:47.031, 2023-11-16 08:45:21.403, … 2023-11-16 09:05:27.379]","[""v_1572055"", ""v_953153"", … ""v_1329494""]"
"""u_425177""","""s_17505104""","[""v_1375331"", ""v_1922852""]","[2, 2]","[2023-11-17 12:42:18.513, 2023-11-17 12:43:52.744]","[""v_2650515"", ""v_367301"", … ""v_676801""]"
"""u_700997""","""s_15528830""","[""v_2152997"", ""v_2152997"", ""v_1217630""]","[2, 1, 2]","[2023-11-16 17:22:53.530, 2023-11-16 17:23:19.885, 2023-11-16 17:23:33.472]","[""v_1431178"", ""v_2395102"", … ""v_1352210""]"
"""u_687201""","""s_11187326""","[""v_2602959"", ""v_549337"", … ""v_212060""]","[2, 2, … 1]","[2023-11-21 14:41:30.546, 2023-11-21 15:00:25.506, … 2023-11-21 15:24:56.271]","[""v_1465910"", ""v_1586766"", … ""v_668735""]"
"""u_573219""","""s_27394215""","[""v_1836184"", ""v_2733253"", … ""v_2297339""]","[1, 2, … 2]","[2023-11-21 18:48:43.424, 2023-11-21 18:49:12.552, … 2023-11-21 19:48:55.704]","[""v_701974"", ""v_1081658"", … ""v_27909""]"
"""u_1082782""","""s_30341164""","[""v_1026036"", ""v_1026036"", … ""v_206700""]","[2, 1, … 2]","[2023-11-17 02:54:19.561, 2023-11-17 02:54:50.486, … 2023-11-17 03:21:22.008]","[""v_964765"", ""v_460169"", … ""v_9584""]"
"""u_81444""","""s_1909964""","[""v_2497384"", ""v_1480870"", … ""v_1500697""]","[2, 2, … 1]","[2023-11-15 12:38:34.579, 2023-11-15 13:03:41.360, … 2023-11-15 13:05:15.916]","[""v_1431178"", ""v_2395102"", … ""v_2192945""]"
